In [2]:
import requests
import zipfile
import os
import datetime
import tarfile


class ZipClass :
    def __init__(self, savingPath, zipUrl, fileName):
        print("ZipClass is initialized")
        self.zipFileName = "my_zip.zip"
        self.savingPath = savingPath
        self.zipUrl = zipUrl
        self.fileInfos = None
        self.fileName = fileName
        self.tgzName = None
    """Downloading the zip file and saving it in the savingPath"""
    def downloadZip(self):
        try : 
            #Data retrieving from the url given
            r = requests.get(self.zipUrl)
            with open(self.savingPath+"/"+self.zipFileName, "wb") as f:
                f.write(r.content)
        except Exception as e:
            print(e)
    
    def extractContent(self):
        try:
            zipRef = zipfile.ZipFile(self.savingPath+"/"+self.zipFileName, 'r')
            zipRef.extractall(self.savingPath)
            zipRef.close()
        except Exception as e:
            print(e)
    def clean(self):
        try : 
            os.remove(self.savingPath + "/" + self.zipFileName)
            os.remove(self.savingPath + "/" + self.fileName)
            #os.remove(self.savingPath + "/" + self.tgzName)
        except Exception as e:
            print(e)
    def checkIfContainsTheFileAndGetInfos(self):
        try:
            zipRef = zipfile.ZipFile(self.savingPath+"/"+self.zipFileName, 'r')
            for zipInfo in zipRef.filelist:
                if(zipInfo.filename == self.fileName):
                    self.fileInfos = zipInfo
                    zipRef.close()
                    return True
            zipRef.close()
            return False
        except Exception as e:
            print(e)
    def checkCorrectDay(self):
        try:
            fileDate = datetime.date(*(self.fileInfos).date_time[0:3])
            todayDate = datetime.date.today()
            return datetime.date.today() == fileDate
        except Exception as e:
            print(e)
    def compressIntoTGZ(self):
        try:
            today = datetime.date.today()
            self.tgzName = ("%d%d%d"%(today.year, today.month, today.day))+".tgz"
            tar = tarfile.open("%s/%s"%(self.savingPath,self.tgzName), "w:gz")
            tar.add(("%s/%s"%(self.savingPath, self.fileName)), arcname=self.fileName)
            tar.close()
        except Exception as e:
            print(e)
    

In [96]:
import re
import json
import smtplib
import ssl
import email.utils
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from email_validator import validate_email
import smtpd
import asyncore
import os
import threading

class EmlServer(smtpd.SMTPServer):
    no = 0
    def process_message(self, peer, mailfrom, rcpttos, data):
        print('Receiving message from:', peer)
        print('Message addressed from:', mailfrom)
        print('Message addressed to:', rcpttos)
        print('Message length:', len(data))
        return



class EmailHandler :
    def __init__(self, config):
        try :
            self.authentication = config["authentication"]
            self.serverIp = config["serverIp"]
            self.serverPort = config["serverPort"]
            self.destinations = config["destinations"]
            self.emailTitle = config["emailTitle"]
            self.attachLog = config["attachLog"]
            self.logFileName = config["logFileName"]
            self.emailHandler = None
            self.localServer = None
            self.initializeService()
        except Exception as e:
            print(e)
    def initializeService(self):
        try:
            context = ssl.create_default_context()
            # Open secure connection with SMTP server to send e-mails
            self.emailHandler = smtplib.SMTP_SSL(
                self.serverIp, port=self.serverPort
            )

        except TimeoutError:
            self.log_email_matt.warning(
                "E-mail server connection",
                "Timeout error, check e-mail server is down. E-mails not sent.",
            )

        except smtplib.SMTPServerDisconnected as e:
            print(e)
    def sendFromLocal(self, linesList):
    #TODO : here we create an internal smtp server : Need to change this in here
        try:
            message = MIMEMultipart()
            message.set_unixfrom('author')
            message['To'] = email.utils.formataddr(('Recipient', self.destinations[0]))
            message['From'] = email.utils.formataddr(('Author', self.authentication["email"]))
            message["Subject"] = self.emailTitle
            message.attach(MIMEText(self.formatEmailContent(linesList)))
            if(self.attachLog):
                message = self.attachLogToMessage(message)
            localServer = smtplib.SMTP('localhost')
            localServer.set_debuglevel(1)
            localServer.send_message(message)
        except Exception as e:
            print(e)
    
    def sendEmail(self, linesList):
        try:
            self.emailHandler.login(self.authentication["email"],self.authentication["password"])
            message = MIMEMultipart()
            message["Subject"] = self.emailTitle
            message["To"] = self.destinations[0]
            print(message["To"])
            message["From"] = self.authentication["email"]
            message.attach(MIMEText(self.formatEmailContent(linesList)))
            if(self.attachLog):
                message = self.attachLogToMessage(message)
            response = self.emailHandler.send_message(message)
        except smtplib.SMTPAuthenticationError:
            print("Logging-in email server , The server did not accept the username/password combination. E-mails not sent.") 
        except smtplib.SMTPNotSupportedError:
            print("Logging-in email server , The AUTH command is not supported by the server. E-mails not sent.")
        # Another error
        except smtplib.SMTPException:
            print("Logging-in email server, Unknown error occured while logging-in to your mail account. E-mail(s)not sent.")
        except Exception as e:
            print(e)
        finally :
            if(self.emailHandler!=None):
                self.emailHandler.quit()
    def formatEmailContent(self, linesList):
        formattedContent = ""
        for line in linesList:
            formattedContent += line + "\n"
        return formattedContent
    def attachLogToMessage(self, message):
        try:
            fileInstance = MIMEBase("application", "octet-stream")
            logFile = open(os.getcwd()+"/"+self.logFileName, "rb")
            fileInstance.set_payload(logFile.read())
            encoders.encode_base64(fileInstance)
            fileInstance.add_header("Content-Disposition", "attachment; filename= " + self.logFileName)
            message.attach(fileInstance)
            return message
        except EnvironmentError:
            print(e)
            print("Attachment, Unknown error occured during mail attachment loading. No attachment sent with e-mails.")
            


In [100]:
import json
from os.path import exists


defaultConfig = {
    "fileName" : "dimpfile.sql",
    "zipUrl" : "http://localhost:7123/webServer/my_zip.zip",
    "loaclServerConfig" : {
        "port" : 9999
    },
    "zipFileName" : "my_zip.zip",
    "smtpConfig" : {
        
    }
}



class ConfigHandler:
    def __init__(self, configPath, configFileName):
        self.config = None
        self.configPath = configPath
        self.configFileName = configFileName
        self.loadConfig()
    def loadConfig(self):
        try:
            jsonFile = open("%s/%s"%(self.configPath, self.configFileName))
            self.config = json.load(jsonFile)
        except Exception as e:
            print(e)
            
    def generateConfig(self):
        try:
            if(self.checkConfigFileExists()):
                os.remove(configFileName)
            with open("config.json", "w") as configFile:
                json.dump(defaultConfig, configFile, indent="\t")
                print("Configuration file regenerated successfully.")    
        except Exception as e:
            print(e)
    def checkIfConfigFileExists(self):
        return exists(self.configFileName)
    def getConfig(self):
        return self.config
    def getSMTPConfig(self):
        smtpConfig = self.config["smtpConfig"]
        smtpConfig["attachLog"] = self.config["attachLog"]
        smtpConfig["logFileName"] = self.config["logFileName"]
        return smtpConfig
    def getLogConfig(self):
        logConfig = {}
        logConfig["logFileName"] = self.config["logFileName"]
        logConfig["attachLog"] = self.config["attachLog"]
        return logConfig
        
    

In [101]:
configHandler = ConfigHandler(os.getcwd(), "config.json")
config = configHandler.getConfig()
print(configHandler.getSMTPConfig())
emailHandler = EmailHandler(configHandler.getSMTPConfig())
emailHandler.sendEmail(["This is an email"])
emailHandler.sendFromLocal(["This is daily report email"])

{'authentication': {'email': 'noreply@ariba.ma', 'password': 'Agadir414$'}, 'serverIp': 'mail.ariba.ma', 'serverPort': 465, 'emailTitle': 'Daily Report', 'destinations': ['mouaadsk@protonmail.com'], 'attachLog': True, 'logFileName': 'log.log'}
mouaadsk@protonmail.com
[Errno 111] Connection refused


In [194]:
import enum 
import logging
from imp import reload

class LogMessageType(enum.Enum):
    Debug = 1
    Info = 2
    Critical = 3
    Warning = 4
    Error = 5
    Exception = 6
    
class LogHandler:
    def __init__(self, config):
        try:
            self.config = config
            #clearing the logFile content
            open("log.log", "w").close()
            self.errorCounts = {"crtitical" : 0,"warning" : 0,"debug" : 0, "error" : 0}
            reload(logging)
            logging.basicConfig(filename="log.log",format="%(asctime)s - %(levelname)s - %(message)s",filemode='a',level=logging.NOTSET)
            self.logger = logging.getLogger("script_logger")
        except Exception as e:
            print(e)
    def addLogLine(self, logMessageType ,logAction , messageText):
        try :
            if logMessageType is LogMessageType.Info:
                msg = "%s : %s" %(logAction ,messageText)
                self.logger.info(msg)
                print("we added infos")
            elif logMessageType is LogMessageType.Warning:
                self.logger.warning("%s : %s" %(logAction ,messageText))
                self.errorCounts["warning"]+=1
            elif logMessageType is LogMessageType.Error:
                self.logger.error("%s : %s" %(logAction ,messageText))
                self.errorCounts["error"]+=1
            elif logMessageType is LogMessageType.Critical:
                self.logger.critical("%s : %s" %(logAction ,messageText))
                self.errorCounts["critical"]+=1
            elif logMessageType is LogMessageType.Debug:
                self.logger.debug("%s : %s" %(logAction ,messageText))
                self.errorCounts["debug"]+=1
        except Exception as e:
            print(e)
    def close(self):
        self.logger.close()

In [195]:
logConfig = configHandler.getLogConfig()
logHandler = LogHandler(logConfig)
logHandler.addLogLine(LogMessageType.Info, "Getting the config File", "Error in getting the config file")

we added infos
